### import

In [15]:
import pandas as pd
import random
import os
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb

### Fixed RandomSeed

In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

### Data Load

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [4]:
train.head()

,ID,생산년도,모델출시년도,브랜드,차량모델명,판매도시,판매구역,주행거리,배기량,압축천연가스(CNG),경유,가솔린,하이브리드,액화석유가스(LPG),가격
0,TRAIN_00000,2018,2014,skoda,fabia,KAT,SLA,85231,999,0,0,1,0,0,51.74
1,TRAIN_00001,2010,2006,toyota,auris,RKO,SWI,135000,1598,0,0,1,0,0,41.47
2,TRAIN_00002,2002,2002,mercedes-benz,clk-klasa,GNI,WIE,255223,1796,0,0,1,0,0,17.81
3,TRAIN_00003,2006,2001,nissan,x-trail,EHX,WIE,238000,2184,0,1,0,0,0,18.20
4,TRAIN_00004,2007,2007,fiat,bravo,OSW,MAL,251000,1910,0,1,0,0,0,17.55


### Split

In [5]:
train_x = train.drop(['ID', '가격'], axis = 1)
train_y = train['가격']

test_x = test.drop('ID', axis = 1)

### Data Processing

In [6]:
ordinal_features = ['브랜드', '차량모델명', '판매도시', '판매구역']

for feature in ordinal_features:
    le = LabelEncoder()
    le = le.fit(train_x[feature])
    train_x[feature] = le.transform(train_x[feature])

    # train데이터에서 존재하지 않았던 값이 test 데이터에 존재할 수도 있습니다.
    # 따라서 test 데이터를 바로 변형시키지 않고 고유값을 확인후 test 데이터를 변환합니다.
    # Data Leakage를 발생시키지 않기 위함이니, 반드시 주의해주세요.
    for label in np.unique(test_x[feature]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    test_x[feature] = le.transform(test_x[feature])

### Train-Validation Split

In [7]:
X_train, X_val, y_train, y_val = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

### LightGBM 데이터셋으로 변환

In [8]:
train_data = lgb.Dataset(X_train, label=y_train)

### Set hyper-parameter grid

In [9]:
param_grid = {
    'num_leaves': [31, 50, 100],
    'learning_rate': [0.05, 0.1, 0.2],
    'n_estimators': [100, 200, 300],
    'reg_alpha': [0, 1, 2],
    'reg_lambda': [0, 1, 2]
}

### LightGBM 모델 생성

In [10]:
lgb_model = lgb.LGBMRegressor()

### 그리드 서치를 사용한 하이퍼파라미터 튜닝

In [11]:
grid_search = GridSearchCV(estimator=lgb_model, param_grid=param_grid, cv=3, scoring='neg_mean_absolute_error')
grid_search.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=LGBMRegressor(),
             param_grid={'learning_rate': [0.05, 0.1, 0.2],
                         'n_estimators': [100, 200, 300],
                         'num_leaves': [31, 50, 100], 'reg_alpha': [0, 1, 2],
                         'reg_lambda': [0, 1, 2]},
             scoring='neg_mean_absolute_error')

### 최적의 하이퍼파라미터 출력

In [12]:
print("Best Hyperparameters:", grid_search.best_params_)

Best Hyperparameters: {'learning_rate': 0.1, 'n_estimators': 300, 'num_leaves': 100, 'reg_alpha': 1, 'reg_lambda': 1}


### 최적의 모델로 재학습

In [13]:
best_model = grid_search.best_estimator_
best_model.fit(X_train, y_train)

LGBMRegressor(n_estimators=300, num_leaves=100, reg_alpha=1, reg_lambda=1)

### 검증 데이터에 대한 예측 수행 및 평가 지표 출력

In [16]:
y_pred = best_model.predict(X_val)

mae = mean_absolute_error(y_val, y_pred)
print("Mean Absolute Error:", mae)

Mean Absolute Error: 5.948773243733233


### Inference & Submit

In [18]:
preds = best_model.predict(test_x)

In [19]:
submission = pd.read_csv('sample_submission.csv')
submission['가격'] = preds
submission.to_csv('./lightgbm_with_gridsearchCV_submit.csv', index = False)